In [1]:
from transformers import BasicTokenizer, Trainer
from datasets import concatenate_datasets, load_from_disk
import kagglehub
import optuna
import torch
import math
import base

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [2]:
base.reset_seed()

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


In [4]:
my_glove = kagglehub.dataset_download("thanakomsn/glove6b300dtxt")
print(my_glove)

/home/jovyan/.cache/kagglehub/datasets/thanakomsn/glove6b300dtxt/versions/1


In [5]:
GLOVE_FILE = f"{my_glove}/glove.6B.300d.txt"
DATASET = "sst2"

In [6]:
train_data = load_from_disk(f"~/data/{DATASET}/train-logits")
eval_data = load_from_disk(f"~/data/{DATASET}/eval-logits")
test_data = load_from_disk(f"~/data/{DATASET}/test-logits")

all_train_data = load_from_disk(f"~/data/{DATASET}/train-logits-augmented")

all_data = concatenate_datasets([load_from_disk(file) for file in [f"~/data/{DATASET}/eval-logits", f"~/data/{DATASET}/test-logits", f"~/data/{DATASET}/train-logits-augmented"]])
tokenizer = BasicTokenizer(do_lower_case=True)

In [7]:
train_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), train_data))
eval_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), eval_data))
test_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), test_data))

all_train_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), all_train_data))

all_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), all_data))

In [8]:
vocab = base.get_vocab(all_data_tokens)

In [9]:
word_index = dict(zip(vocab, range(len(vocab))))

In [10]:
embeddings_index = base.get_embeddings_indeces(GLOVE_FILE)

Found 400000 word vectors.


In [11]:
print(len(vocab))
num_tokens = len(vocab) + 2
embedding_dim = 300

14621


In [12]:
embedding_matrix = base.get_embedding_matrix(num_tokens, embedding_dim, word_index, embeddings_index)

Converted 14305 words (316) misses


In [13]:
train_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),train_data_tokens))
eval_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),eval_data_tokens))
test_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),test_data_tokens))

all_train_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),all_train_data_tokens))

In [14]:
train_padded_data = list(map(lambda x: base.padd(x,60), train_data_index))
eval_padded_data = list(map(lambda x: base.padd(x,60), eval_data_index))
test_padded_data = list(map(lambda x: base.padd(x,60), test_data_index))

all_train_padded_data = list(map(lambda x: base.padd(x,60), all_train_data_index))

In [15]:
train_data = train_data.add_column("input_ids", train_padded_data)
eval_data = eval_data.add_column("input_ids", eval_padded_data)
test_data = test_data.add_column("input_ids", test_padded_data)

all_train_data = all_train_data.add_column("input_ids", all_train_padded_data)

In [16]:
num_epochs = 15
batch_size = 128

In [17]:
#Nápočet epoch na steps
data_length = len(train_data)
min_r = math.ceil(data_length/batch_size)*5
max_r = math.ceil(data_length/batch_size)*num_epochs
warm_up = math.ceil(data_length/batch_size/10)

In [18]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up)
    }   
    print(f"Trial {trial.number} with params: {params}")
    return params

In [19]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [20]:
def get_BiLSTM():
    return base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=2, freeze_embed=False)

In [21]:
base.reset_seed()

In [22]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-base-embedd_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-base-embedd_hp-search", epochs=num_epochs, batch_size=batch_size)

In [23]:
trainer = Trainer(
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM()
)
  

In [24]:
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Base-embedd",
    n_trials=150
)

[I 2025-03-23 01:16:23,863] A new study created in memory with name: Base-embedd


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 32}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.385600,0.407834,0.814220,0.821123,0.815547,0.813607
2,0.242800,0.461820,0.815367,0.821979,0.814021,0.813916
3,0.184700,0.481049,0.829128,0.833275,0.828082,0.828232
4,0.148400,0.618027,0.810780,0.822933,0.808969,0.808309
5,0.121700,0.552868,0.830275,0.830239,0.830176,0.830203
6,0.100500,0.640374,0.832569,0.834543,0.831839,0.832060
7,0.084300,0.635275,0.836009,0.836348,0.835680,0.835828
8,0.072800,0.660173,0.826835,0.829696,0.827682,0.826669
9,0.062000,0.724519,0.830275,0.832481,0.831018,0.830167
10,0.053600,0.779018,0.819954,0.824356,0.821009,0.819629


[I 2025-03-23 01:18:16,235] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.0007875660249889869, 'weight_decay': 0.001, 'warmup_steps': 6}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.296600,0.427054,0.834862,0.837631,0.835691,0.834715
2,0.138000,0.492721,0.833716,0.835139,0.833091,0.833310
3,0.080400,0.455078,0.838303,0.838287,0.838185,0.838226
4,0.048700,0.697592,0.840596,0.840844,0.840311,0.840443
5,0.032300,0.904633,0.830275,0.830275,0.830386,0.830261
6,0.021600,1.046377,0.824541,0.826332,0.823830,0.824030
7,0.015800,1.165881,0.831422,0.831862,0.831050,0.831209
8,0.009300,1.210838,0.830275,0.830460,0.830008,0.830124
9,0.006600,1.427148,0.829128,0.829451,0.828797,0.828939
10,0.004300,1.802318,0.825688,0.825629,0.825629,0.825629


[I 2025-03-23 07:06:07,054] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.0012924272883759233, 'weight_decay': 0.005, 'warmup_steps': 19}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.314100,0.523554,0.808486,0.826698,0.810632,0.806470
2,0.164500,0.435669,0.840596,0.840582,0.840479,0.840521
3,0.106200,0.405227,0.856651,0.856632,0.856750,0.856636
4,0.071900,0.602238,0.841743,0.843326,0.841100,0.841339
5,0.049900,0.770074,0.830275,0.830321,0.830092,0.830167
6,0.034600,0.831368,0.834862,0.836856,0.834133,0.834360
7,0.024500,0.922542,0.832569,0.832848,0.832260,0.832396
8,0.016100,1.222348,0.837156,0.837156,0.837270,0.837142
9,0.012600,1.178339,0.832569,0.832568,0.832428,0.832481
10,0.009400,1.475344,0.825688,0.826775,0.825124,0.825321


Using the latest cached version of the module from /home/jovyan/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--recall/11f90e583db35601050aed380d48e83202a896976b9608432fba9244fb447f24 (last modified on Fri Jan 10 23:14:00 2025) since it couldn't be found locally at evaluate-metric--recall, or remotely on the Hugging Face Hub.
[I 2025-03-23 07:08:23,826] Trial 149 pruned. 


In [25]:
print(best_trial)

BestRun(run_id='12', objective=0.8439702128779307, hyperparameters={'learning_rate': 0.002163019453168294, 'weight_decay': 0.006, 'warmup_steps': 41}, run_summary=None)


In [26]:
base.reset_seed()

In [27]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-distill-embedd_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-distill-embedd_hp-search", remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

In [28]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

In [29]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [30]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM(),
)
  

In [31]:
best_trial2 = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill-embedd",
    n_trials=150
)

[I 2025-03-23 07:08:24,142] A new study created in memory with name: Distill-embedd


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 32, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.720400,1.466995,0.823394,0.825325,0.824093,0.823302
2,0.924000,1.401038,0.831422,0.832830,0.830797,0.831011
3,0.648300,1.526375,0.829128,0.837538,0.827661,0.827566
4,0.492900,1.503149,0.822248,0.823406,0.821662,0.821854
5,0.382500,1.508689,0.841743,0.841801,0.841564,0.841642


[I 2025-03-23 07:09:22,825] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.00010255552094216992, 'weight_decay': 0.0, 'warmup_steps': 38, 'lambda_param': 0.6000000000000001, 'temperature': 5.5}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.588200,1.580865,0.806193,0.812320,0.807453,0.805628
2,0.788700,1.350542,0.831422,0.833764,0.830629,0.830843
3,0.514800,1.453244,0.847477,0.850878,0.846563,0.846823
4,0.370300,1.661969,0.822248,0.834305,0.820483,0.820021
5,0.281500,1.437249,0.846330,0.848175,0.845647,0.845901
6,0.222800,1.325038,0.854358,0.854861,0.853993,0.854173
7,0.183000,1.379640,0.856651,0.857454,0.856203,0.856420
8,0.153000,1.369805,0.855505,0.855470,0.855582,0.855486
9,0.128200,1.426454,0.852064,0.853753,0.852709,0.852008
10,0.111200,1.458320,0.860092,0.860758,0.860508,0.860085


[I 2025-03-23 15:20:02,475] Trial 148 finished with value: 0.8577075931043558 and parameters: {'learning_rate': 0.00041567747760923817, 'weight_decay': 0.007, 'warmup_steps': 36, 'lambda_param': 0.7000000000000001, 'temperature': 3.5}. Best is trial 80 with value: 0.8646333249136988.


Trial 149 with params: {'learning_rate': 0.002093527576873113, 'weight_decay': 0.001, 'warmup_steps': 30, 'lambda_param': 1.0, 'temperature': 7.0}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.188100,1.489000,0.834862,0.844677,0.836406,0.834077
2,0.431600,1.068162,0.860092,0.860828,0.859666,0.859879
3,0.234000,1.133332,0.854358,0.855320,0.853867,0.854095
4,0.149800,1.166203,0.856651,0.857623,0.856161,0.856393
5,0.103800,1.333348,0.841743,0.841712,0.841648,0.841676
6,0.079000,1.195067,0.852064,0.852334,0.851783,0.851922
7,0.060700,1.213841,0.852064,0.852334,0.851783,0.851922
8,0.049500,1.214607,0.857798,0.857848,0.857961,0.857791
9,0.042800,1.260999,0.847477,0.847961,0.847110,0.847284
10,0.037400,1.234280,0.847477,0.847573,0.847278,0.847371


[I 2025-03-23 15:23:21,188] Trial 149 pruned. 


In [32]:
print(best_trial2)

BestRun(run_id='80', objective=0.8646333249136988, hyperparameters={'learning_rate': 0.0006819636125122306, 'weight_decay': 0.01, 'warmup_steps': 30, 'lambda_param': 0.9, 'temperature': 3.0}, run_summary=None)


In [33]:
#Nápočet epoch na steps
data_length = len(all_train_data)
min_r = math.ceil(data_length/batch_size)*5
max_r = math.ceil(data_length/batch_size)*num_epochs
warm_up = math.ceil(data_length/batch_size/10)

In [34]:
base.reset_seed()

In [35]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-base-embedd_aug_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-base-embedd_aug_hp-search", epochs=num_epochs, batch_size=batch_size)

In [36]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

In [37]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [38]:
trainer = Trainer(
    args=training_args,
    train_dataset=all_train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM(),
)
  

In [ ]:
best_trial3 = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Base-aug-embedd",
    n_trials=150
)

[I 2025-03-23 15:23:21,625] A new study created in memory with name: Base-aug-embedd


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 305}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.193800,0.532892,0.846330,0.847017,0.845900,0.846096
2,0.079600,0.677365,0.833716,0.833795,0.833891,0.833710
3,0.050900,0.889196,0.826835,0.828421,0.826166,0.826372
4,0.035400,1.003145,0.826835,0.826787,0.826882,0.826807
5,0.025900,1.285178,0.827982,0.827982,0.828092,0.827967


[I 2025-03-23 15:33:46,156] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.0007875660249889869, 'weight_decay': 0.001, 'warmup_steps': 65}


...

Trial 49 with params: {'learning_rate': 5.3252681664836025e-05, 'weight_decay': 0.007, 'warmup_steps': 162}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.303200,0.435733,0.815367,0.816486,0.814779,0.814958
2,0.179000,0.496302,0.827982,0.827925,0.828008,0.827949
3,0.135000,0.668016,0.809633,0.815886,0.808306,0.808176
4,0.111200,0.690091,0.825688,0.827847,0.824914,0.825113
5,0.094700,0.731090,0.833716,0.833696,0.833807,0.833698
6,0.083400,0.798178,0.830275,0.830273,0.830134,0.830186
7,0.074900,0.801212,0.824541,0.824478,0.824503,0.824489
8,0.067800,0.841250,0.826835,0.826785,0.826755,0.826769
9,0.061900,0.942137,0.831422,0.831360,0.831428,0.831385
10,0.057200,0.962445,0.823394,0.823329,0.823377,0.823349


[I 2025-03-24 07:30:20,731] Trial 49 finished with value: 0.8175547376579559 and parameters: {'learning_rate': 5.3252681664836025e-05, 'weight_decay': 0.007, 'warmup_steps': 162}. Best is trial 30 with value: 0.8324621660744835.


Trial 50 with params: {'learning_rate': 0.00020965151071803784, 'weight_decay': 0.003, 'warmup_steps': 218}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.206300,0.527965,0.846330,0.846876,0.845942,0.846123


In [ ]:
print(best_trial3)

In [ ]:
base.reset_seed()

In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-distill-embedd_aug_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-distill-embedd_aug_hp-search", remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

In [ ]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

In [ ]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

In [ ]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=all_train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM(),
)
  

In [ ]:
best_trial4 = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill-aug-embedd",
    n_trials=150
)

In [ ]:
print(best_trial4)

In [ ]:
print("Best normal training score: ", best_trial)
print("Best distilation trianing score: ", best_trial2)
print("Best normal training score with augmentations: ", best_trial3)
print("Best distilation trianing score with augmentations: ",best_trial4)